In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QCheckBox, QHBoxLayout, QWidget, QVBoxLayout, QLabel, QPushButton
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer
import numpy as np
import cv2
from collections import deque

class Desenare(QWidget):
    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        self.setWindowTitle('Desenare')

        self.video_label = QLabel(self)
        self.mask_label = QLabel(self)
        button = QPushButton('Sterge', self)
        button.clicked.connect(self.reset_paint)

        self.drawing_checkbox = QCheckBox('Deseneaza', self)
        self.drawing_checkbox.setChecked(False)  
        self.drawing_checkbox.stateChanged.connect(self.toggle_drawing)
       
        label_layout = QHBoxLayout()
        label_layout.addWidget(self.video_label)
        label_layout.addWidget(self.mask_label)

        self.layout = QVBoxLayout()
        self.layout.addLayout(label_layout) 
        self.layout.addWidget(self.drawing_checkbox)
        self.layout.addWidget(button)

        self.setLayout(self.layout)

        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)

        self.init_opencv()
        self.show()
        self.drawing_allowed = False
        
    def init_opencv(self):
        self.cap = cv2.VideoCapture(0)

        cv2.namedWindow("Trackbars")
        cv2.createTrackbar("highH", "Trackbars", 126, 180, self.nimic)
        cv2.createTrackbar("lowH", "Trackbars", 87, 180, self.nimic)
        cv2.createTrackbar("highS", "Trackbars", 255, 255, self.nimic)
        cv2.createTrackbar("lowS", "Trackbars", 88, 255, self.nimic)
        cv2.createTrackbar("highV", "Trackbars", 255, 255, self.nimic)
        cv2.createTrackbar("lowV", "Trackbars", 18, 255, self.nimic)

        self.color = [deque(maxlen=1024)]
        self.color_index = 0
        self.kernel = np.ones((5, 5), np.uint8)
        self.penColor = [(255, 0, 0)]
        self.colorIndex = 0

        self.timer.start(30)

    def nimic(self, x):
        pass
    
    
    def toggle_drawing(self, state):
        if state == 2:
            self.drawing_allowed = True
        else:
            self.drawing_allowed = False
    
    def reset_paint(self):
        self.color = [deque(maxlen=512)]
        self.color_index = 0

    def update_frame(self):
        ret, frame = self.cap.read()
        frame = cv2.flip(frame, 1)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        high_hue = cv2.getTrackbarPos("highH", "Trackbars")
        low_hue = cv2.getTrackbarPos("lowH", "Trackbars")
        high_saturation = cv2.getTrackbarPos("highS", "Trackbars")
        low_saturation = cv2.getTrackbarPos("lowS", "Trackbars")
        high_value = cv2.getTrackbarPos("highV", "Trackbars")
        low_value = cv2.getTrackbarPos("lowV", "Trackbars")

        high_hsv = np.array([high_hue, high_saturation, high_value])
        low_hsv = np.array([low_hue, low_saturation, low_value])

        Mask = cv2.inRange(hsv, low_hsv, high_hsv)
        Mask = cv2.erode(Mask, self.kernel, iterations=1)
        Mask = cv2.morphologyEx(Mask, cv2.MORPH_OPEN, self.kernel)
        Mask = cv2.dilate(Mask, self.kernel, iterations=1)

        cnts, _ = cv2.findContours(Mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        center = None

        if len(cnts) > 0:
            cnt = sorted(cnts, key=cv2.contourArea, reverse=True)[0]
            ((x, y), radius) = cv2.minEnclosingCircle(cnt)

            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)

            M = cv2.moments(cnt)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

            if 50 <= center[0] <= 150:
                self.color = [deque(maxlen=512)]
                self.color_index = 0
            else:
                if self.colorIndex == 0:
                    self.color[self.color_index].appendleft(center)
                else:
                    self.color.append(deque(maxlen=512))
                    self.color_index += 1
     
        if self.drawing_allowed == True:
            line = [self.color]
            for i in range(len(line)):
                for j in range(len(line[i])):
                    for k in range(1, len(line[i][j])):
                        cv2.line(frame, line[i][j][k - 1], line[i][j][k], self.penColor[i], 2)

        h, w = frame.shape[:2]
        combined_image = np.zeros((h, 2 * w, 3), dtype=np.uint8)
        combined_image[:, :w] = frame
        combined_image[:, w:] = cv2.cvtColor(Mask, cv2.COLOR_GRAY2BGR)

        video_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        video_image = QImage(video_image.data, video_image.shape[1], video_image.shape[0], QImage.Format_RGB888)
        video_pixmap = QPixmap.fromImage(video_image)
        self.video_label.setPixmap(video_pixmap)

        mask_image = cv2.cvtColor(Mask, cv2.COLOR_GRAY2RGB)
        mask_image = QImage(mask_image.data, mask_image.shape[1], mask_image.shape[0], QImage.Format_RGB888)
        mask_pixmap = QPixmap.fromImage(mask_image)
        self.mask_label.setPixmap(mask_pixmap)

    def closeEvent(self, event):
        self.cap.release()

def main():
    app = QApplication(sys.argv)
    ex = Desenare()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

C:\Users\justf\AppData\Local\Temp\ipykernel_6024\2672561635.py:9: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class DrawingApp(QWidget):


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


SystemExit: 0

C:\Users\justf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
